In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [5]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)  # Padding agregado
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0)
        self.conv3 = nn.Conv2d(16, 120, kernel_size=5, stride=1, padding=0)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.avg_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.avg_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv3(x))
        x = x.view(-1, 120)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [6]:
# Transformaciones para normalizar los datos
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Descargar y cargar los datos de entrenamiento y prueba
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet5().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # Suma la pérdida de batch
            pred = output.argmax(dim=1, keepdim=True)  # Obtiene el índice de la clase con la mayor probabilidad
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.316202
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.830585
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.260402
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.226408
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.221488
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.244046
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.108628
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.250745
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.219912
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.123928
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.023017
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.104196
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.051343
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.019677
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.091084
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.152854
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.035083
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.071048
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.137233
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.066952
Train Epoch: